In [1]:
import torch
import torchvision as tv
import os
from transformer_flow import Model
import utils
import pathlib

os.environ["CUDA_VISIBLE_DEVICES"] = "4" # set GPU

utils.set_random_seed(100)
notebook_output_path = pathlib.Path('runs/notebook')

In [2]:
dataset = 'mnist'
num_classes = 10
img_size = 28
channel_size = 1

# we use a small model for fast demonstration, increase the model size for better results
patch_size = 4
channels = 128
blocks = 4
layers_per_block = 4
# try different noise levels to see its effect
noise_std = 0.1

batch_size = 256
lr = 5e-4
# increase epochs for better results
epochs = 100
sample_freq = 5

if torch.cuda.is_available():
    device = 'cuda' 
elif torch.backends.mps.is_available():
    device = 'mps' # if on mac
else:
    device = 'cpu' # if mps not available
print(f'using device {device}')

# fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channel_size * patch_size ** 2, device=device)
fixed_noise = torch.randn(num_classes * 10, (img_size // patch_size)**2, channels, device=device)
fixed_y = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, 10).flatten()

transform = tv.transforms.Compose([
    tv.transforms.Resize((img_size, img_size)),
    tv.transforms.ToTensor(),
    tv.transforms.Normalize((0.5,), (0.5,))
])
data = tv.datasets.MNIST('.', transform=transform, train=True, download=True)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=True)

model = Model(in_channels=channel_size, img_size=img_size, patch_size=patch_size, 
              channels=channels, num_blocks=blocks, layers_per_block=layers_per_block,
              num_classes=num_classes).to(device)

optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.95), lr=lr, weight_decay=1e-4)
lr_schedule = utils.CosineLRSchedule(optimizer, len(data_loader), epochs * len(data_loader), 1e-6, lr)

model_name = f'{patch_size}_{channels}_{blocks}_{layers_per_block}_{noise_std:.2f}_sqa'
sample_dir = notebook_output_path / f'{dataset}_samples_{model_name}'
ckpt_file = notebook_output_path / f'{dataset}_model_{model_name}.pth'
sample_dir.mkdir(exist_ok=True, parents=True)

all_S = []

for epoch in range(epochs):
    losses = 0
    for x, y in data_loader:
        x = x.to(device)
        eps = noise_std * torch.randn_like(x)
        x = x + eps
        y = y.to(device)
        optimizer.zero_grad()
        z, outputs, logdets = model(x, y)
        loss = model.get_loss(z, logdets)
        loss.backward()
        optimizer.step()
        lr_schedule.step()
        losses += loss.item()

        # print the singular values of the matrix of the x embedder
        try:
          W = model.x_embedder.weight
          S = torch.linalg.svdvals(W, driver='gesvd')
          all_S.append(S)
        except:
            print(f"there is error in calculating the singular values")
            print(f"matrix: {W.shape}")
            print(f"matrix: {W}")
            print(f"loss: {loss}")
            loss=torch.tensor(float('nan'), device=device)
        if torch.isnan(loss): 
            for i in range(1,6):
                print(f'singular value of S[-{i}]: {all_S[-i]}')
            assert False
          
        # print(f'singular values {S.tolist()}')

    print(f"epoch {epoch} lr {optimizer.param_groups[0]['lr']:.6f} loss {losses / len(data_loader):.4f}")
    print('layer norms', ' '.join([f'{z.pow(2).mean():.4f}' for z in outputs]))
    print(f'logdet: {logdets.mean():.4f}, prior p: {0.5 * z.pow(2).mean():.4f}')
    if (epoch + 1) % sample_freq == 0:
        with torch.no_grad():
            samples = model.reverse(fixed_noise, fixed_y)
        tv.utils.save_image(samples, sample_dir / f'samples_{epoch:03d}.png', normalize=True, nrow=10)
        tv.utils.save_image(model.unpatchify(z[:100]), sample_dir / f'latent_{epoch:03d}.png', normalize=True, nrow=10)
        print('sampling complete')
torch.save(model.state_dict(), ckpt_file)

using device cuda
Number of parameters: 3.40M
epoch 0 lr 0.000500 loss -1.3205
layer norms 0.4940 0.7211 0.7874 0.8999
logdet: 2.2685, prior p: 0.4499
epoch 1 lr 0.000500 loss -2.1360
layer norms 0.3807 0.3023 0.5149 0.8692
logdet: 2.8488, prior p: 0.4346
epoch 2 lr 0.000499 loss -2.6566
layer norms 0.3484 0.2866 0.5893 0.9638
logdet: 3.3940, prior p: 0.4819
epoch 3 lr 0.000499 loss -2.9701
layer norms 0.3266 0.2756 0.6159 0.7621
logdet: 3.6050, prior p: 0.3811
epoch 4 lr 0.000498 loss -3.0766
layer norms 0.3485 0.3472 0.8748 0.8166
logdet: 3.5887, prior p: 0.4083
sampling complete
epoch 5 lr 0.000497 loss -3.2266
layer norms 0.3592 0.3515 0.9865 0.7930
logdet: 3.6820, prior p: 0.3965
epoch 6 lr 0.000495 loss -3.4265
layer norms 0.3544 0.4040 1.0694 1.2317
logdet: 4.1122, prior p: 0.6158
epoch 7 lr 0.000494 loss -3.5050
layer norms 0.3744 0.4504 1.2230 0.7969
logdet: 3.7841, prior p: 0.3984
epoch 8 lr 0.000492 loss -3.5892
layer norms 0.3312 0.4237 1.3407 2.0380
logdet: 4.3340, prior p

AssertionError: x has nan after block reverse

In [ ]:
# now we can also evaluate the model by turning it into a classifier with Bayes rule, p(y|x) = p(y)p(x|y)/p(x)
data = tv.datasets.MNIST('.', transform=transform, train=False, download=False)
data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True, drop_last=False)
num_correct = 0
num_examples = 0
for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    eps = noise_std * torch.randn_like(x)
    x = x.repeat(num_classes, 1, 1, 1)
    y_ = torch.arange(num_classes, device=device).view(-1, 1).repeat(1, y.size(0)).flatten()
    with torch.no_grad():
        z, outputs, logdets = model(x, y_)
        losses = 0.5 * z.pow(2).mean(dim=[1, 2]) - logdets # keep the batch dimension
        pred = losses.reshape(num_classes, y.size(0)).argmin(dim=0)
    num_correct += (pred == y).sum()
    num_examples += y.size(0)
print(f'Accuracy %{100 * num_correct / num_examples:.2f}')

Accuracy %98.35
